In [128]:
import pandapower as pp
import pandapower.networks as pn
import pandapower.topology as top
import numpy as np

# Load the IEEE 30-bus system
net = pn.case30()

# Run the initial power flow
pp.runpp(net)
print("Initial power flow run successful.")

# Define function to check for N-1 contingency
def check_n_1_contingency(net):
    critical_elements = []
    
    # Backup original network
    original_net = net.deepcopy()
    
    # Test line outages
    for line in net.line.index:
        net_copy = original_net.deepcopy()
        net_copy.line.at[line, "in_service"] = False  # Remove one line
        try:
            pp.runpp(net_copy, algorithm="nr")
        except pp.powerflow.LoadflowNotConverged:
            critical_elements.append(f"Line {line} outage causes failure.")
            continue
        
        # Check for overloads
        if any(net_copy.res_line.loading_percent > 100):
            critical_elements.append(f"Line {line} outage causes overloads.")

    # Test generator outages
    for gen in net.gen.index:
        net_copy = original_net.deepcopy()
        net_copy.gen.at[gen, "in_service"] = False  # Remove one generator
        try:
            pp.runpp(net_copy, algorithm="nr")
        except pp.powerflow.LoadflowNotConverged:
            critical_elements.append(f"Generator {gen} outage causes failure.")
            continue

        # Check for voltage violations
        if any((net_copy.res_bus.vm_pu < 0.95) | (net_copy.res_bus.vm_pu > 1.05)):
            critical_elements.append(f"Generator {gen} outage causes voltage issues.")

    return critical_elements

# Run the N-1 contingency check
violations = check_n_1_contingency(net)

# If there are issues
if violations:
    print("N-1 Contingency Violations Detected:")
    for v in violations:
        print(v)

else:
    print("The IEEE 30-bus system satisfies the N-1 contingency criterion.")

Initial power flow run successful.
N-1 Contingency Violations Detected:
Line 0 outage causes overloads.
Line 1 outage causes overloads.
Line 2 outage causes overloads.
Line 3 outage causes overloads.
Line 4 outage causes overloads.
Line 5 outage causes overloads.
Line 6 outage causes overloads.
Line 7 outage causes overloads.
Line 8 outage causes overloads.
Line 9 outage causes overloads.
Line 10 outage causes overloads.
Line 11 outage causes overloads.
Line 12 outage causes overloads.
Line 13 outage causes overloads.
Line 14 outage causes overloads.
Line 15 outage causes overloads.
Line 16 outage causes overloads.
Line 17 outage causes overloads.
Line 18 outage causes overloads.
Line 19 outage causes overloads.
Line 20 outage causes overloads.
Line 21 outage causes overloads.
Line 22 outage causes overloads.
Line 23 outage causes overloads.
Line 24 outage causes overloads.
Line 25 outage causes overloads.
Line 26 outage causes overloads.
Line 27 outage causes overloads.
Line 28 outage

In [130]:
import pandapower as pp
import pandapower.networks as pn
import numpy as np

# Load the IEEE 30-bus system
net = pn.case30()

# Function to reinforce the network based on contingency violations
def reinforce_network(net):
    original_net = net.deepcopy()
    
    # Identify problematic lines
    overloaded_lines = []
    for line in net.line.index:
        net_copy = original_net.deepcopy()
        net_copy.line.at[line, "in_service"] = False  # Simulate line outage
        try:
            pp.runpp(net_copy)
        except pp.powerflow.LoadflowNotConverged:
            overloaded_lines.append(line)
            continue
        if any(net_copy.res_line.loading_percent > 100):
            overloaded_lines.append(line)
    
    # Add parallel lines to overloaded lines
    for line in overloaded_lines:
        from_bus = net.line.loc[line, "from_bus"]
        to_bus = net.line.loc[line, "to_bus"]
        print(f"Adding parallel line between Bus {from_bus} and Bus {to_bus} to mitigate overload.")
        pp.create_line_from_parameters(net, from_bus=from_bus, to_bus=to_bus, 
                                       length_km=1.0, r_ohm_per_km=0.05, 
                                       x_ohm_per_km=0.1, c_nf_per_km=0, 
                                       max_i_ka=1.5)  # Higher capacity
    
    # Identify problematic generators
    problematic_gens = []
    for gen in net.gen.index:
        net_copy = original_net.deepcopy()
        net_copy.gen.at[gen, "in_service"] = False  # Simulate generator outage
        try:
            pp.runpp(net_copy)
        except pp.powerflow.LoadflowNotConverged:
            problematic_gens.append(gen)
            continue
        if any((net_copy.res_bus.vm_pu < 0.95) | (net_copy.res_bus.vm_pu > 1.05)):
            problematic_gens.append(gen)

    # Increase generator capacities to provide redundancy
    for gen in problematic_gens:
        net.gen.at[gen, "p_mw"] *= 1.2  # Increase power generation by 20%
        print(f"Increasing capacity of Generator {gen} to improve voltage stability.")

    return net

# Reinforce the network
net = reinforce_network(net)

# Run contingency check again
pp.runpp(net)
print("Reinforced system power flow successful.")

# Check if violations still exist
violations = check_n_1_contingency(net)
if violations:
    print("System still has issues under N-1 contingency.")
    for v in violations:
        print(v)
else:
    print("The 30-bus system satisfies the N-1 contingency criterion!")

Adding parallel line between Bus 0 and Bus 1 to mitigate overload.
Adding parallel line between Bus 0 and Bus 2 to mitigate overload.
Adding parallel line between Bus 1 and Bus 3 to mitigate overload.
Adding parallel line between Bus 2 and Bus 3 to mitigate overload.
Adding parallel line between Bus 1 and Bus 4 to mitigate overload.
Adding parallel line between Bus 1 and Bus 5 to mitigate overload.
Adding parallel line between Bus 3 and Bus 5 to mitigate overload.
Adding parallel line between Bus 4 and Bus 6 to mitigate overload.
Adding parallel line between Bus 5 and Bus 6 to mitigate overload.
Adding parallel line between Bus 5 and Bus 7 to mitigate overload.
Adding parallel line between Bus 5 and Bus 8 to mitigate overload.
Adding parallel line between Bus 5 and Bus 9 to mitigate overload.
Adding parallel line between Bus 8 and Bus 10 to mitigate overload.
Adding parallel line between Bus 8 and Bus 9 to mitigate overload.
Adding parallel line between Bus 3 and Bus 11 to mitigate ove